In [1]:
# get imports
import os
import json
import numpy as np
import pandas as pd
import plotly.graph_objects as go

In [2]:
# load in data
model_name = '2020_08_24-OD1599_NU'

In [3]:
# generate csvs as model folder in separate steps
folderpath = os.path.join(os.getcwd(), 'workspace', model_name)

step_names = [
    '1_compiled_data', 
    '2_compiled_data_no_outliers', 
    '3_compiled_data_interpolation', 
    None, 
    '5_compiled_data_warped',
    '6_cell_coordinates_by_timepoint',
    '7_cell_coordinates_by_timepoint_smoothed'
]

csv_output_folder = os.path.join('workspace', model_name, 'step_csvs')
for step_idx, step_name in enumerate(step_names):
    if not step_name:
        continue
        
    step_output_folder = os.path.join(csv_output_folder, step_name)
    
    filepath_model_csv = os.path.join(folderpath, step_name + '.json')
    with open(filepath_model_csv) as f:
        data_model_csv = json.load(f)
    
    for strain in data_model_csv.keys():
        strain_folder = os.path.join(step_output_folder, strain)
        
        # steps 1-5
        if step_idx < 5:
            for pos_name in data_model_csv[strain].keys():
                pos_folder = os.path.join(strain_folder, pos_name)

                if not os.path.exists(pos_folder):
                    os.makedirs(pos_folder)

                for cell_type in ['seam_cells', 'annotations']:
                    cell_coords = data_model_csv[strain][pos_name][cell_type]

                    for cell_name_iter in cell_coords.keys():
                        filename_csv = cell_name_iter +'.csv'
                        filepath_csv = os.path.join(pos_folder, filename_csv)

                        coordinates = cell_coords[cell_name_iter]['coordinates']
                        timepoints = cell_coords[cell_name_iter]['timepoints']
                        new_table = pd.DataFrame(coordinates)
                        new_table.columns = ['x', 'y', 'z']
                        new_table.insert(0, "timepoints", timepoints, True)

                        new_table.to_csv(filepath_csv, index=False)
        else: # steps 6-7
            folder = os.path.join(step_output_folder, 'combined_model_step_{}'.format(step_idx+1))
            
            if not os.path.exists(folder):
                os.makedirs(folder)
                    
            for cell_type in ['seam_cells', 'annotations']:
                cell_coords = data_model_csv[cell_type]

                for cell_name_iter in cell_coords.keys():
                    filename_csv = cell_name_iter +'.csv'
                    filepath_csv = os.path.join(folder, filename_csv)

                    coordinates = cell_coords[cell_name_iter]
                    timepoints = np.arange(0, len(coordinates))
                    new_table = pd.DataFrame(coordinates)
                    new_table.columns = ['x', 'y', 'z']
                    new_table.insert(0, "timepoints", timepoints, True)

                    new_table.to_csv(filepath_csv, index=False)